# Dataset fields description
fields:
- Maker: Producer name 
- Genmodel: Model name
- Genmodel_ID: Unique model ID
- Adv_ID: Advertisement_ID - unique for each row
- Adv_Year: Advertisement year
- Adv_Month: Advertisement month
- Color:
- Reg_Year: Registration year
- Bodytype:
- Mileage:
- Engine_Size: Engine capacity in liters
- Gearbox: type of the gearbox (category)
- Fuel_Type:
- Price:
- Engine_Power:
- Annual_Tax:
- Wheelbase:
- Height:
- Width:
- Length:
- Average_Mpg: miles per gallon
- Top_Speed: mph
- Seat_Num:
- Door_Num


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('capstone_data.csv')

### Renaming columns with typos etc.

In [3]:
names = df.columns.values

for i in range(len(names)):
    names[i] = names[i].strip().capitalize()
    if "_" in names[i]:
        split1 = names[i].split("_")[0]
        split2 = names[i].split("_")[1]
        if split2.lower() == "id":
            split2 = split2.upper()
        else:
            split2 = split2.capitalize()      
        names[i] = split1 + "_" + split2

df = df.set_axis(labels=names, axis="columns")

In [4]:
df.rename(columns={"Engin_Size": "Engine_Size", "Runned_Miles": "Mileage", "Bodytype": "Body_Type", "Gearbox": "Gearbox_Type"}, inplace=True)
df.columns

Index(['Maker', 'Genmodel', 'Genmodel_ID', 'Adv_ID', 'Adv_Year', 'Adv_Month',
       'Color', 'Reg_Year', 'Body_Type', 'Mileage', 'Engine_Size',
       'Gearbox_Type', 'Fuel_Type', 'Price', 'Engine_Power', 'Annual_Tax',
       'Wheelbase', 'Height', 'Width', 'Length', 'Average_Mpg', 'Top_Speed',
       'Seat_Num', 'Door_Num'],
      dtype='object')

## Pre-cleaning

In [5]:
df.dtypes

Maker            object
Genmodel         object
Genmodel_ID      object
Adv_ID           object
Adv_Year          int64
Adv_Month         int64
Color            object
Reg_Year        float64
Body_Type        object
Mileage          object
Engine_Size      object
Gearbox_Type     object
Fuel_Type        object
Price           float64
Engine_Power    float64
Annual_Tax       object
Wheelbase       float64
Height          float64
Width           float64
Length          float64
Average_Mpg      object
Top_Speed        object
Seat_Num        float64
Door_Num        float64
dtype: object

### Handling null values in wrong format

In [6]:
df = df.replace("nan", np.nan)
df[["Length", "Height", "Width", "Wheelbase"]] = df[["Length", "Height", "Width", "Wheelbase"]].replace(0, np.nan)

### Replacing $$ in Adv_ID 

In [7]:
def dolars(x):
    if "$$" in x:
        x = x.replace("$$", "_")
    return x

df["Adv_ID"] = df["Adv_ID"].apply(dolars)
df.head()

,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_Year,Adv_Month,Color,Reg_Year,Body_Type,Mileage,...,Engine_Power,Annual_Tax,Wheelbase,Height,Width,Length,Average_Mpg,Top_Speed,Seat_Num,Door_Num
0,Bentley,Arnage,10_1,10_1_1,2018,4,Silver,2000.0,Saloon,60000,...,NaN,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0
1,Bentley,Arnage,10_1,10_1_2,2018,6,Grey,2002.0,Saloon,44000,...,450.0,315,3116.0,1515.0,2125.0,5390.0,13.7 mpg,179 mph,5.0,4.0
2,Bentley,Arnage,10_1,10_1_3,2017,11,Blue,2002.0,Saloon,55000,...,400.0,315,3116.0,1515.0,2125.0,5390.0,14.7 mpg,155 mph,5.0,4.0
3,Bentley,Arnage,10_1,10_1_4,2018,4,Green,2003.0,Saloon,14000,...,NaN,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0
4,Bentley,Arnage,10_1,10_1_5,2017,11,Grey,2003.0,Saloon,61652,...,NaN,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0


### Deleting rows with Adv_Month greater than 12

In [8]:
df[df["Adv_Month"] > 12]

,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_Year,Adv_Month,Color,Reg_Year,Body_Type,Mileage,...,Engine_Power,Annual_Tax,Wheelbase,Height,Width,Length,Average_Mpg,Top_Speed,Seat_Num,Door_Num
46560,Ford,Streetka,29_32,29_32_75,2012,13,Blue,2005.0,Convertible,22000,...,94.0,NaN,2450.0,1332.0,1852.0,3650.0,35.8 mpg,108 mph,2.0,2.0
135473,Peugeot,407,69_23,69_23_2,2012,17,Blue,2006.0,Coupe,77000,...,205.0,NaN,2725.0,1399.0,1868.0,4815.0,33.2 mpg,143 mph,4.0,2.0
154764,Rolls-Royce,Phantom,77_4,77_4_10,2012,33,Black,2007.0,Saloon,20000,...,NaN,555,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN


In [9]:
df.drop(df[df["Adv_Month"] > 12].index, inplace=True)

In [10]:
def to_month_abr(x):
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    if str(x).isnumeric():
        x = months[x - 1]
    return x

In [11]:
df["Adv_Month"] = df["Adv_Month"].apply(to_month_abr).astype("category")

### Removing units or non-numerical characters from numeric columns

In [12]:
def remove_units(x):
    if x is not np.nan:
        output = ""
        x = str(x)
        for i in range(len(x)):
            if x[i].isnumeric() or x[i] == ".":
                output += x[i]
        if output == "":
            x = np.nan
        else:
            x = float(output)
    return x

In [13]:
df["Engine_Size"] = df["Engine_Size"].apply(remove_units)
df["Average_Mpg"] = df["Average_Mpg"].apply(remove_units)
df["Top_Speed"] = df["Top_Speed"].apply(remove_units)
df["Mileage"] = df["Mileage"].apply(remove_units)
df["Annual_Tax"] = df["Annual_Tax"].apply(remove_units)
df.dtypes

Maker             object
Genmodel          object
Genmodel_ID       object
Adv_ID            object
Adv_Year           int64
Adv_Month       category
Color             object
Reg_Year         float64
Body_Type         object
Mileage          float64
Engine_Size      float64
Gearbox_Type      object
Fuel_Type         object
Price            float64
Engine_Power     float64
Annual_Tax       float64
Wheelbase        float64
Height           float64
Width            float64
Length           float64
Average_Mpg      float64
Top_Speed        float64
Seat_Num         float64
Door_Num         float64
dtype: object

### Genmodel - managing data-like formats

In [14]:
df["Genmodel"].iloc[187981]

'09-Mar'

In [15]:
df["Genmodel"].iloc[188688]

'09-May'

In [16]:
df["Genmodel"] = df["Genmodel"].replace("09-Mar", "9-3")
df["Genmodel"] = df["Genmodel"].replace("09-May", "9-5")

### Engine_Size - correcting outliers

In [17]:
df[df["Engine_Size"].isnull() == False]["Engine_Size"].sort_values().tail()

146118       8.2
194214      10.0
130712    1400.0
114114    2998.0
130799    3500.0
Name: Engine_Size, dtype: float64

In [18]:
def eng_size_outliers(x):
    if x > 1000:
        x = x/1000
    return x

df["Engine_Size"] = df["Engine_Size"].apply(eng_size_outliers)

### Fuel_Types - merging duplicate categories

In [19]:
df["Fuel_Type"].unique()

array(['Petrol', 'Diesel', 'Petrol Plug-in Hybrid', 'Petrol Ethanol',
       'Hybrid  Petrol/Electric Plug-in', 'Electric', nan,
       'Hybrid  Diesel/Electric', 'Bi Fuel', 'Hybrid  Petrol/Electric',
       'Petrol Hybrid', 'Diesel Hybrid', 'Diesel Plug-in Hybrid',
       'Hybrid  Diesel/Electric Plug-in'], dtype=object)

In [20]:
df["Fuel_Type"] = df["Fuel_Type"].apply(lambda x: str(x).replace("  ", " "))
df["Fuel_Type"].unique()

array(['Petrol', 'Diesel', 'Petrol Plug-in Hybrid', 'Petrol Ethanol',
       'Hybrid Petrol/Electric Plug-in', 'Electric', 'nan',
       'Hybrid Diesel/Electric', 'Bi Fuel', 'Hybrid Petrol/Electric',
       'Petrol Hybrid', 'Diesel Hybrid', 'Diesel Plug-in Hybrid',
       'Hybrid Diesel/Electric Plug-in'], dtype=object)

In [21]:
df["Fuel_Type"] = df["Fuel_Type"].replace("Hybrid Petrol/Electric Plug-in", "Petrol Plug-in Hybrid") 
df["Fuel_Type"] = df["Fuel_Type"].replace("Hybrid Petrol Plug-in", "Petrol Plug-in Hybrid")
df["Fuel_Type"] = df["Fuel_Type"].replace("Hybrid Diesel Plug-in", "Diesel Plug-in Hybrid")
df["Fuel_Type"] = df["Fuel_Type"].replace("Hybrid Diesel/Electric Plug-in", "Diesel Plug-in Hybrid")
df["Fuel_Type"] = df["Fuel_Type"].replace("Hybrid Diesel/Electric", "Diesel Hybrid")
df["Fuel_Type"] = df["Fuel_Type"].replace("Hybrid Petrol/Electric", "Petrol Hybrid")
# There should be 10 resulting categories including NaN

In [22]:
df["Fuel_Type"] = df["Fuel_Type"].replace("nan", np.nan)
df["Fuel_Type"].unique()

array(['Petrol', 'Diesel', 'Petrol Plug-in Hybrid', 'Petrol Ethanol',
       'Electric', nan, 'Diesel Hybrid', 'Bi Fuel', 'Petrol Hybrid',
       'Diesel Plug-in Hybrid'], dtype=object)

### Removing outliers

In [23]:
df[df["Price"] > 3000000]

,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_Year,Adv_Month,Color,Reg_Year,Body_Type,Mileage,...,Engine_Power,Annual_Tax,Wheelbase,Height,Width,Length,Average_Mpg,Top_Speed,Seat_Num,Door_Num
104958,McLaren,675LT,58_6,58_6_13,2017,Jun,Silver,2016.0,Coupe,200.0,...,666.0,555.0,2670.0,1188.0,2095.0,4546.0,NaN,205.0,2.0,2.0


In [24]:
df.drop(df[df["Price"] > 3000000].index, inplace=True)


In [25]:
df["Reg_Year"].sort_values().head()

237059    1900.0
21120     1960.0
141874    1960.0
21114     1970.0
148255    1970.0
Name: Reg_Year, dtype: float64

In [26]:
df.drop(df["Reg_Year"].sort_values().head(1).index, inplace=True)

### Correcting data types

In [27]:
df.dtypes

Maker             object
Genmodel          object
Genmodel_ID       object
Adv_ID            object
Adv_Year           int64
Adv_Month       category
Color             object
Reg_Year         float64
Body_Type         object
Mileage          float64
Engine_Size      float64
Gearbox_Type      object
Fuel_Type         object
Price            float64
Engine_Power     float64
Annual_Tax       float64
Wheelbase        float64
Height           float64
Width            float64
Length           float64
Average_Mpg      float64
Top_Speed        float64
Seat_Num         float64
Door_Num         float64
dtype: object

In [28]:
df.nunique()

Maker               88
Genmodel           896
Genmodel_ID        881
Adv_ID          268250
Adv_Year             9
Adv_Month           12
Color               22
Reg_Year            25
Body_Type           18
Mileage          72420
Engine_Size         72
Gearbox_Type         3
Fuel_Type            9
Price            20454
Engine_Power       466
Annual_Tax          24
Wheelbase          426
Height             578
Width              475
Length             983
Average_Mpg        358
Top_Speed          138
Seat_Num            10
Door_Num             7
dtype: int64

In [29]:
df["Adv_Year"] = df["Adv_Year"].astype("category")
df["Gearbox_Type"] = df["Gearbox_Type"].astype("category")
df["Fuel_Type"] = df["Fuel_Type"].astype("category")

### Reordering columns

In [30]:
col_names = df.columns
col_names

Index(['Maker', 'Genmodel', 'Genmodel_ID', 'Adv_ID', 'Adv_Year', 'Adv_Month',
       'Color', 'Reg_Year', 'Body_Type', 'Mileage', 'Engine_Size',
       'Gearbox_Type', 'Fuel_Type', 'Price', 'Engine_Power', 'Annual_Tax',
       'Wheelbase', 'Height', 'Width', 'Length', 'Average_Mpg', 'Top_Speed',
       'Seat_Num', 'Door_Num'],
      dtype='object')

In [31]:
new_order = ['Maker', 'Genmodel', 'Genmodel_ID', 'Adv_ID', 'Adv_Year', 'Adv_Month', 'Color', 'Body_Type', 'Gearbox_Type', 'Fuel_Type', 'Reg_Year', 'Mileage', 'Engine_Size','Price', 'Engine_Power', 'Annual_Tax', 'Wheelbase', 'Height', 'Width', 'Length', 'Average_Mpg', 'Top_Speed', 'Seat_Num', 'Door_Num']
len(new_order)

24

In [32]:
df = df.reindex(new_order, axis="columns")
df.head()

,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_Year,Adv_Month,Color,Body_Type,Gearbox_Type,Fuel_Type,...,Engine_Power,Annual_Tax,Wheelbase,Height,Width,Length,Average_Mpg,Top_Speed,Seat_Num,Door_Num
0,Bentley,Arnage,10_1,10_1_1,2018,Apr,Silver,Saloon,Automatic,Petrol,...,NaN,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0
1,Bentley,Arnage,10_1,10_1_2,2018,Jun,Grey,Saloon,Automatic,Petrol,...,450.0,315.0,3116.0,1515.0,2125.0,5390.0,13.7,179.0,5.0,4.0
2,Bentley,Arnage,10_1,10_1_3,2017,Nov,Blue,Saloon,Automatic,Petrol,...,400.0,315.0,3116.0,1515.0,2125.0,5390.0,14.7,155.0,5.0,4.0
3,Bentley,Arnage,10_1,10_1_4,2018,Apr,Green,Saloon,Automatic,Petrol,...,NaN,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0
4,Bentley,Arnage,10_1,10_1_5,2017,Nov,Grey,Saloon,Automatic,Petrol,...,NaN,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0


In [33]:
df.to_csv("/Users/mpolcyn/PycharmProjects/DataProcessingCapstone/capstone_data_clean.csv", index=False)